<a href="https://colab.research.google.com/github/jmparradov/EACD-03-BIGDATA/blob/master/06_%5BTALLER%5D_DataFrame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/UDEA-Esp-Analitica-y-Ciencia-de-Datos/EACD-03-BIGDATA/master/init.py
import init; init.init(force_download=False);
from IPython.display import Image

replicating local resources


In [ ]:
Image("local/imgs/udea-datascience.png")

In [13]:
#Instalación
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!pip install -q findspark

#Variables de Entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"



In [42]:
#SparkContext
import findspark
import re

from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext

from pyspark.sql.types import IntegerType
from pyspark.sql.functions import to_timestamp, regexp_replace, lower, when
from pyspark.sql.types import StringType

# Adding Context
findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()
sqlCtx = SQLContext(sc)

# **TALLER 2: DataFrame**
## Desarrollado por: Mg. Efraín Alberto Oviedo; Ing. Kevin Martínez Gallego
## Universidad de Antioquia

Por favor, ingrese la información de cada integrante del grupo de trabajo

**Nombre Completo:** Juan Mauricio Parrado Villa

**Cédula:** 1017206626

**Correo Institucional:** mauricio.parrado@udea.edu.co

#**Análisis de becas académicas para estudiantes Colombianos**

El ICETEX ha dispuesto un histórico de becas ofrecidas para estudios de nivel de pregrado y posgrado, presenciales o virtuales, que han sido propuestas para fortalecer la formación académica de ciudadanos colombianos en el exterior.

(Disponible en https://www.datos.gov.co/Educaci-n/Icetex-Hist-rico-de-Becas/eb6r-dfgr)

Para cada caso se reportan los siguientes datos:

* Código Convocatoria
* Nombre del Oferente: Nombre del gobierno que ofrece la beca.
* Área del Conocimiento
* Nombre del Programa
* Nombre del País
* Fecha de Recepción -mm/dd/aaaa
* Perfil del Aplicante
* Objetivo del Programa: Breve descripción del programa académico.

Se solicita de su ayuda para responder a los siguientes interrogantes:

In [17]:
df_Becas = sqlCtx.read.option("header",True).csv("local/data/icetex_becas.csv")
df_Becas.show()

+-------------------+--------------------+---------------------+--------------------+--------------------+------------------------------+--------------------+---------------------+
|Codigo Convocatoria| Nombre del Oferente|Area del Conocimiento| Nombre del programa|     Nombre del País|Fecha de Recepción -mm/dd/aaaa|Perfil del Aplicante|Objetivo del Programa|
+-------------------+--------------------+---------------------+--------------------+--------------------+------------------------------+--------------------+---------------------+
|          2127916.0|OEA - TOP - ARGEN...|       ADMINISTRACIÓN|POLÍTICAS DE GOBI...|PLATAFORMA DE EDU...|          05/27/2016 12:00:...|El programa está ...| a) Proporcionar a...|
|          3101616.0|ALIANZA DEL PACÍF...|     DIFERENTES AREAS|PLATAFORMA DE MOV...|Chile (República ...|                          null|El programa está ...| El objetivo del P...|
|          2221414.0|GOBIERNO DE LA INDIA|  INGENIERIA Y AFINES|DATA COMMUNICATIO...|India (Rep

##**A) Calidad de datos**

Revise cada una de las variables disponibles y resuelva los problemas de calidad de datos que puedan afectar el desarrollo de este taller, justificando las razones de su decisión para cada caso


Presente aquí el código desarrollado y el resultado obtenido

In [18]:
# Cambiando los títulos para evitar espacios
for col in df_Becas.columns:
    df_Becas = df_Becas.withColumnRenamed(col, col.lower().replace(" ", "_"))
df_Becas = df_Becas.withColumnRenamed("fecha_de_recepción_-mm/dd/aaaa", "fecha_de_recepción")

# Cast 'codigo_convocatoria' to integer
df_Becas = df_Becas.withColumn("codigo_convocatoria", df_Becas["codigo_convocatoria"].cast(IntegerType()))

# Cast 'fecha_de_recepción' to timestamp
df_Becas = df_Becas.withColumn("fecha_de_recepción", to_timestamp("fecha_de_recepción", "MM/dd/yyyy hh:mm a"))
df_Becas.show()

+-------------------+--------------------+---------------------+--------------------+--------------------+------------------+--------------------+---------------------+
|codigo_convocatoria| nombre_del_oferente|area_del_conocimiento| nombre_del_programa|     nombre_del_país|fecha_de_recepción|perfil_del_aplicante|objetivo_del_programa|
+-------------------+--------------------+---------------------+--------------------+--------------------+------------------+--------------------+---------------------+
|            2127916|OEA - TOP - ARGEN...|       ADMINISTRACIÓN|POLÍTICAS DE GOBI...|PLATAFORMA DE EDU...|              null|El programa está ...| a) Proporcionar a...|
|            3101616|ALIANZA DEL PACÍF...|     DIFERENTES AREAS|PLATAFORMA DE MOV...|Chile (República ...|              null|El programa está ...| El objetivo del P...|
|            2221414|GOBIERNO DE LA INDIA|  INGENIERIA Y AFINES|DATA COMMUNICATIO...|India (República ...|              null|"El Programa ""Co...| Ofrecer 

In [45]:
# Limpiando los datos
def remove_quotes_from_strings(df_Becas):
    for col in df_Becas.columns:
        if df_Becas.schema[col].dataType == StringType():
            df_Becas = df_Becas.withColumn(col, regexp_replace(df_Becas[col], '"', ''))
    return df_Becas

def remove_parentheses_content(text):
  return re.sub(r'\([^)]*\)', '', text)

df_Becas = remove_quotes_from_strings(df_Becas)
df_Becas = df_Becas.withColumn("nombre_del_país", regexp_replace(df_Becas["nombre_del_país"], r'\([^)]*\)', ''))
df_Becas = df_Becas.withColumn("nombre_del_país", when(lower(df_Becas["nombre_del_país"]).contains("plataforma"), "Plataforma Virtual").otherwise(df_Becas["nombre_del_país"]))
df_Becas.show()

+-------------------+--------------------+---------------------+--------------------+------------------+------------------+--------------------+---------------------+
|codigo_convocatoria| nombre_del_oferente|area_del_conocimiento| nombre_del_programa|   nombre_del_país|fecha_de_recepción|perfil_del_aplicante|objetivo_del_programa|
+-------------------+--------------------+---------------------+--------------------+------------------+------------------+--------------------+---------------------+
|            2127916|OEA - TOP - ARGEN...|       ADMINISTRACIÓN|POLÍTICAS DE GOBI...|Plataforma Virtual|              null|El programa está ...| a) Proporcionar a...|
|            3101616|ALIANZA DEL PACÍF...|     DIFERENTES AREAS|PLATAFORMA DE MOV...|            Chile |              null|El programa está ...| El objetivo del P...|
|            2221414|GOBIERNO DE LA INDIA|  INGENIERIA Y AFINES|DATA COMMUNICATIO...|            India |              null|El Programa Comun...| Ofrecer a los par...

In [46]:
df_Becas.createOrReplaceTempView("Becas")
result = spark.sql("""
    SELECT * FROM Becas
    WHERE LOWER(`nombre_del_país`) like '%plataforma%'
""")
result.show()


+-------------------+--------------------+---------------------+--------------------+------------------+------------------+--------------------+---------------------+
|codigo_convocatoria| nombre_del_oferente|area_del_conocimiento| nombre_del_programa|   nombre_del_país|fecha_de_recepción|perfil_del_aplicante|objetivo_del_programa|
+-------------------+--------------------+---------------------+--------------------+------------------+------------------+--------------------+---------------------+
|            2127916|OEA - TOP - ARGEN...|       ADMINISTRACIÓN|POLÍTICAS DE GOBI...|Plataforma Virtual|              null|El programa está ...| a) Proporcionar a...|
|            2106413|    OEA - TOP - PAEC|       ADMINISTRACIÓN|EVALUACIÓN INSTIT...|Plataforma Virtual|              null|El programa está ...| Brindar a los par...|
|            2126016|   OEA - FONDO VERDE|    CIENCIAS SOCIALES|DIPLOMADO EN EXPE...|Plataforma Virtual|              null|El programa está ...| Promover los méto...

In [ ]:
# prompt: from df_Becas replace the values with "plataforma" converting this field in lowercase, with Plataforma

from pyspark.sql.functions import


df_Becas.show()

##**B) Aumento de ofertas de becas**

Identifique si la cantidad de becas ofrecidas para ciudadanos colombianos se ha incrementado con los años

Presente aquí el código desarrollado y el resultado obtenido

##**C) Paises que han ofrecido becas**

Presente un listado con todos los países (diferentes de Colombia) de los que se tiene reporte que han ofrecido becas para estudiantes colombianos (*solamente presentar nombres de países, y estos solo deben aparecer una única vez en la lista*)

Presente aquí el código desarrollado y el resultado obtenido

##**D) Áreas del conocimiento y Posgrados**

Se desea averiguar cúal es el área de conocimiento predominante por país. Además, queremos conocer qué porcentaje de los programas ofrecidos (**de acuerdo al nombre del programa**) corresponden a estudios de posgrado.

Presente aquí el código desarrollado y el resultado obtenido

##**E) ¿Qué podemos concluir a partir del Perfil del Aspirante y el Objetivo del Programa?**

Se dispone de dos campos de descripción, que no emplean un formato claro que permita analizar los perfiles de los aspirantes con respecto al objetivo del programa.

Plantee una forma de analizar esta información para conocer las relaciones entre los perfiles y lo objetivos.

Presente aquí el código desarrollado y el resultado obtenido

##**Análisis de los resultados**

Teniendo en cuenta los resultados obtenidos en los puntos anteriores, presente sus impresiones sobre las becas académicas disponibles para los estudiantes colombianos.


#**Notas:**

* Analice los resultados de cada punto y presente sus conclusiones
* Utilice al menos una vez cada uno de los siguientes métodos

  *   agg
  *   withColumn
  *   groupBy
  *   sql
  *   registerFunction